# SCADA Anomaly Detection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import statsmodels.tsa.stattools as sts
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
import xgboost as xgb
from mpl_toolkits.mplot3d import Axes3D
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go

from math import sqrt
import warnings
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)

In [ ]:
data_with_attacks = pd.read_csv('data/01_Lev_fault_Temp_corr_seed_11_vars_23.csv')

## Cleaning Data

In [ ]:
cols = list(data_with_attacks.columns)
cols.append(cols.pop(cols.index('ATTACK')))

In [ ]:
data_with_attacks = data_with_attacks.loc[:,cols]

In [ ]:
data = data_with_attacks[data_with_attacks.ATTACK == 0]

In [ ]:
data.head(2)

In [ ]:
data_with_attacks.head(2)

## Testing for Stationarity

The Augmented Dickey Fuller Test (ADF) is unit root test for stationarity. Unit roots can cause unpredictable results in your time series analysis.

The hypotheses for the test:
* The null hypothesis for this test is that there is a unit root.
* The alternate hypothesis differs slightly according to which equation you’re using. The basic alternate is that the time series is stationary (or trend-stationary).

In [ ]:
l = []
alpha = 0.05

for i in range(1,data.shape[1]-1):
    test  = sts.adfuller(data.iloc[:,i].diff(1).dropna())    
    if(test[1] < alpha):
        is_stationary = True
    else:
        is_stationary = False
        
    row = {}
    row['colname'] = data.columns[i]
    row['is_stationary'] = is_stationary
    l.append(row)

stationarity_tests = pd.DataFrame(l)

In [ ]:
stationarity_tests

In [ ]:
stationarity_tests.to_csv('output/stationarity_tests.csv', index = False)

## IQR Anomaly Detection

### Calculating range for outliers

* The IQR is the length of the box in your box-and-whisker plot. An outlier is any value that lies more than one and a half times the length of the box from either end of the box.

In [ ]:
# function to calculate lower bound
def lower_bound(column):
    return column.min() - 1.5*(column.quantile(0.75) - column.quantile(0.25))

# function to calculate upper bound
def upper_bound(column):
    return column.max() + 1.5*(column.quantile(0.75) - column.quantile(0.25))

In [ ]:
l = []

# find lower and upper bound of every column except first and the last
for i in range(1, data.shape[1] - 1):
    row = {}
    row['column'] = data.columns[i]
    row['upper_bound'] = data.iloc[:,i].agg(upper_bound)
    row['lower_bound'] = data.iloc[:,i].agg(lower_bound)
    l.append(row)
    
iqr_df = pd.DataFrame(l)

In [ ]:
iqr_df.head(5)

In [ ]:
iqr_df.to_csv('output/iqr/iqr_test.csv', index = False)

### Testing outliers on data with attacks

In [ ]:
# function to check if outlier lies between lower and upper bounds
def iqr_outlier_detection(x):
    x = pd.DataFrame(x).reset_index()
    x.columns = ['column', 'value']
    iqr_x = pd.merge(x, iqr_df, on = 'column', how = 'right')
    iqr_x['Test'] = (iqr_x['value'] > iqr_x['upper_bound']) | (iqr_x['value'] < iqr_x['lower_bound'])
    return 1 if iqr_x['Test'].any() else 0

In [ ]:
data_with_attacks['iqr_result'] = data_with_attacks.apply(lambda x: iqr_outlier_detection(x), axis = 1)

### Evaluating IQR Model

In [ ]:
confusion_matrix(y_true = data_with_attacks['ATTACK'], y_pred = data_with_attacks['iqr_result'])

In [ ]:
accuracy_score(y_true = data_with_attacks['ATTACK'], y_pred = data_with_attacks['iqr_result']) * 100

In [ ]:
print(classification_report(y_true = data_with_attacks['ATTACK'], y_pred = data_with_attacks['iqr_result']))

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_true = data_with_attacks['ATTACK'], 
                                  y_score = data_with_attacks['iqr_result'])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_true = data_with_attacks['ATTACK'].ravel(), 
                                          y_score = data_with_attacks['iqr_result'].ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure(dpi = 120)
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.grid()
plt.show()

## Preparing Data for Regression Algorithms

In [ ]:
train_test = data_with_attacks.drop('Time', axis = 1)

In [ ]:
train, test = train_test_split(train_test, test_size=0.25, stratify = train_test['ATTACK'], random_state=1234)

In [ ]:
training, validation = train_test_split(train, test_size=0.25, stratify=train['ATTACK'], random_state=1234)

In [ ]:
x_train = training.drop('ATTACK', axis = 1)
y_train = training['ATTACK']
x_val = validation.drop('ATTACK', axis = 1)
y_val = validation['ATTACK']

In [ ]:
X_test = test.drop('ATTACK', axis = 1)
Y_test = test['ATTACK']

### CatBoostRegressor

In [ ]:
# initializing model with parameters
cbr = CatBoostRegressor(max_depth=4, 
                        learning_rate=0.06900,
                        loss_function='RMSE',
                        n_estimators=500,
                        one_hot_max_size=2,
                        eval_metric='AUC',
                        boosting_type='Ordered',
                        random_seed=2405, 
                        use_best_model=True,
                        silent=True,
                        random_strength=1.0944250924022183,
                        reg_lambda=1.2762795426592872)

In [ ]:
cbr.fit(x_train, y_train, eval_set=(x_val, y_val),plot=True)

#### Evaluation Performance of CatBoostRegressor

In [ ]:
prediction = cbr.predict(X_test)

In [ ]:
sqrt(mean_squared_error(Y_test, prediction))

In [ ]:
class_prediction = prediction.copy()

class_prediction[class_prediction >= 0.5] = 1
class_prediction[class_prediction < 0.5] = 0

In [ ]:
confusion_matrix(y_true = Y_test, y_pred = class_prediction)

In [ ]:
accuracy_score(y_true = Y_test, y_pred = class_prediction) * 100

In [ ]:
print(classification_report(y_true = Y_test, y_pred = class_prediction))

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_true = Y_test, 
                                  y_score = class_prediction)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_true = Y_test.ravel(), 
                                          y_score = class_prediction.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure(dpi = 120)
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.grid()
plt.show()

### XGBoost

In [ ]:
xgc = xgb.XGBClassifier(
        n_estimators=10000,
        objective='binary:logistic',
        n_jobs=-1
    )

In [ ]:
dtrain = xgb.DMatrix(x_train,label=y_train)
dtest = xgb.DMatrix(x_val)

In [ ]:
#setting parameters for XGBoost
parameters={'max_depth':4, 
            'min_child_weight': 18, 
            'objective':'binary:logistic',
            'eval_metric':'auc',
            'learning_rate':0.0638840541928953,
            'subsample': 0.7642763922160356,
            'colsample_bytree': 0.5695032486938503,
            'reg_lambda': 98.64817753125035,}

In [ ]:
xg = xgb.train(parameters,dtrain)

#### Evaluating Performance of XBGoost

In [ ]:
ypred = xg.predict(dtest)

In [ ]:
# performance on validation set
sqrt(mean_squared_error(y_val, ypred))

In [ ]:
# performance on test set
dtest_final = xgb.DMatrix(X_test)

In [ ]:
pred = xg.predict(dtest_final)

In [ ]:
sqrt(mean_squared_error(Y_test, pred))

In [ ]:
class_prediction = pred.copy()

class_prediction[class_prediction >= 0.5] = 1
class_prediction[class_prediction < 0.5] = 0

In [ ]:
confusion_matrix(y_true = Y_test, y_pred = class_prediction)

In [ ]:
accuracy_score(y_true = Y_test, y_pred = class_prediction) * 100

In [ ]:
print(classification_report(y_true = Y_test, y_pred = class_prediction))

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_true = Y_test, 
                                  y_score = class_prediction)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_true = Y_test.ravel(), 
                                          y_score = class_prediction.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure(dpi = 120)
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.grid()
plt.show()

## Isolation Forests

In [ ]:
#specify the column names to be modelled

to_model_columns=data_with_attacks.columns[1:-2]
clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
clf.fit(data_with_attacks[to_model_columns])

pred = clf.predict(data_with_attacks[to_model_columns])
data_with_attacks['isolation_forest_anomaly']=pred

outliers=data_with_attacks.loc[data_with_attacks['isolation_forest_anomaly']==-1]
outlier_index=list(outliers.index)

#Find the number of anomalies and normal points here points classified -1 are anomalous
print(data_with_attacks['isolation_forest_anomaly'].value_counts())

### Plotting outliers and inliers using PCA

Normalize and fit the metrics to a PCA to reduce the number of dimensions and then plot them in 3D highlighting the anomalies.

In [ ]:
pca = PCA(n_components=3)  # Reduce to k=3 dimensions
scaler = StandardScaler()
#normalize the metrics
X = scaler.fit_transform(data_with_attacks[to_model_columns])
X_reduce = pca.fit_transform(X)

In [ ]:
fig = plt.figure(dpi = 200)
ax = fig.add_subplot(111, projection='3d')
ax.set_zlabel("x_composite_3")
# Plot the compressed data points
ax.scatter(X_reduce[:, 0], X_reduce[:, 1], zs=X_reduce[:, 2], s=4, lw=1, label="inliers",c="green")
# Plot x's for the ground truth outliers
ax.scatter(X_reduce[outlier_index,0],X_reduce[outlier_index,1], X_reduce[outlier_index,2],
           lw=2, s=60, marker="x", c="red", label="outliers")
ax.legend()
plt.show()

In [ ]:
data_with_attacks['isolation_forest_result'] = data_with_attacks\
.isolation_forest_anomaly.apply(lambda x: 1 if x == -1 else 0)

### Evaluating Performance of Isolation Forest

In [ ]:
confusion_matrix(y_true = data_with_attacks['ATTACK'], y_pred = data_with_attacks['isolation_forest_result'])

In [ ]:
accuracy_score(y_true = data_with_attacks['ATTACK'], 
               y_pred = data_with_attacks['isolation_forest_result'])* 100

In [ ]:
print(classification_report(y_true = data_with_attacks['ATTACK'], 
                            y_pred = data_with_attacks['isolation_forest_result']))

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_true = data_with_attacks['ATTACK'], 
                                  y_score = data_with_attacks['isolation_forest_result'])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_true = data_with_attacks['ATTACK'].ravel(), 
                                          y_score = data_with_attacks['isolation_forest_result'].ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure(dpi = 120)
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.grid()
plt.show()

* Now we have figured the anomalous behavior at a use case level.
* But to be actionable on the anomaly its important to identify and provide information on which metrics are anomalous in it individually.
* The anomalies identified by the algorithm should make sense when viewed visually(sudden dip/peaks) by the business user to act upon it. 
* So creating a good visualization is equally important in this process.

### Individual Metrics

* This function creates actuals plot on a time series with anomaly points highlighted on it. Also a table which provides actual data, the change and conditional formatting based on anomalies.
* A helper function to find percentage change,classify anomaly based on severity.
* The predict function classifies the data as anomalies based on the results from decision function on crossing a threshold.
* Say if the business needs to find the next level of anomalies which might have an impact, this could be used to identify those points.
* The top 12 quantiles are identified anomalies(high severity), based on decision function here we identify the 12–24 quantile points and classify them as low severity anomalies.

In [ ]:
def plot_anomaly(df, metric_name):
    dates = df.Time
    # identify the anomaly points and create a array of its values for plot
    bool_array = (abs(df['anomaly']) > 0)
    actuals = df["actuals"][-len(bool_array):]
    anomaly_points = bool_array * actuals
    anomaly_points[anomaly_points == 0] = np.nan
    # A dictionary for conditional format table based on anomaly
    color_map = {0: "'rgba(228, 222, 249, 0.65)'", 1: "yellow", 2: "red"}

    # Table which includes Date,Actuals,Change occured from previous point
    table = go.Table(
        domain=dict(x=[0, 1],
                    y=[0, 0.3]),
        columnwidth=[1, 2],
        # columnorder=[0, 1, 2,],
        header=dict(height=20,
                    values=[['<b>Date</b>'], ['<b>Actual Values </b>'], ['<b>% Change </b>'],
                            ],
                    font=dict(color=['rgb(45, 45, 45)'] * 5, size=14),
                    fill=dict(color='#d562be')),
        cells=dict(values=[df.round(3)[k].tolist() for k in ['Time', 'actuals', 'percentage_change']],
                   line=dict(color='#506784'),
                   align=['center'] * 5,
                   font=dict(color=['rgb(40, 40, 40)'] * 5, size=12),
                   # format = [None] + [",.4f"] + [',.4f'],
                   # suffix=[None] * 4,
                   suffix=[None] + [''] + [''] + ['%'] + [''],
                   height=27,
                   fill=dict(color=[test_df['anomaly_class'].map(color_map)],  # map based on anomaly level from dictionary
                             )
                   ))
    # Plot the actuals points
    Actuals = go.Scatter(name='Actuals',
                         x=dates,
                         y=df['actuals'],
                         xaxis='x1', yaxis='y1',
                         mode='line',
                         marker=dict(size=12,
                                     line=dict(width=1),
                                     color="blue"))
    # Highlight the anomaly points
    anomalies_map = go.Scatter(name="Anomaly",
                               showlegend=True,
                               x=dates,
                               y=anomaly_points,
                               mode='markers',
                               xaxis='x1',
                               yaxis='y1',
                               marker=dict(color="red",
                                           size=11,
                                           line=dict(
                                               color="red",
                                               width=2)))
    axis = dict(
        showline=True,
        zeroline=False,
        showgrid=True,
        mirror=True,
        ticklen=4,
        gridcolor='#ffffff',
        tickfont=dict(size=10))
    layout = dict(
        width=1000,
        height=865,
        autosize=False,
        title=metric_name,
        margin=dict(t=75),
        showlegend=True,
        xaxis1=dict(axis, **dict(domain=[0, 1],
                                 anchor='y1', showticklabels=True)),
        yaxis1=dict(axis, **dict(domain=[2 * 0.21 + 0.20, 1], anchor='x1', hoverformat='.2f')))
    fig = go.Figure(data=[table, anomalies_map, Actuals], layout=layout)
    plot(fig, filename='output/isolation_forest/{0}.html'.format(metric_name), auto_open=False)


In [ ]:
def classify_anomalies(df, metric_name):
    df['metric_name'] = metric_name
    df = df.sort_values(by='Time', ascending=False)
    # Shift actuals by one Time to find the percentage chage between current and previous data point
    df['shift'] = df['actuals'].shift(-1)
    df['percentage_change'] = (
        (df['actuals'] - df['shift']) / df['actuals']) * 100
    # Categorise anomalies as 0-no anomaly, 1- low anomaly , 2 - high anomaly
    df['anomaly'].loc[df['anomaly'] == 1] = 0
    df['anomaly'].loc[df['anomaly'] == -1] = 2
    df['anomaly_class'] = df['anomaly']
    max_anomaly_score = df['score'].loc[df['anomaly_class'] == 2].max()
    medium_percentile = df['score'].quantile(0.24)
    df['anomaly_class'].loc[(df['score'] > max_anomaly_score) & (
        df['score'] <= medium_percentile)] = 1
    return df

* Plots for each individual metrics saved as HTML file in plots folder

In [ ]:
for i in range(1, len(data_with_attacks.columns)-4):
    clf.fit(data_with_attacks.iloc[:, i:i+1])
    pred = clf.predict(data_with_attacks.iloc[:, i:i+1])
    test_df = pd.DataFrame()
    test_df['Time'] = data_with_attacks['Time']
    
    # Find decision function to find the score and classify anomalies
    test_df['score'] = clf.decision_function(data_with_attacks.iloc[:, i:i+1])
    test_df['actuals'] = data_with_attacks.iloc[:, i:i+1]
    test_df['anomaly'] = pred
    
    test_df = classify_anomalies(test_df, data_with_attacks.columns[i])
    plot_anomaly(test_df, data_with_attacks.columns[i])

## Graphical Representation of IQR

In [ ]:
def plot_iqr_anomaly(df, metric_name):
    dates = df.Time
    # identify the anomaly points and create a array of its values for plot
    bool_array = (abs(df['anomaly']) > 0)
    actuals = df["actuals"][-len(bool_array):]
    anomaly_points = bool_array * actuals
    anomaly_points[anomaly_points == 0] = np.nan
    # A dictionary for conditional format table based on anomaly
    color_map = {0: "'rgba(228, 222, 249, 0.65)'", 1: "red"}

    # Table which includes Date,Actuals,Change occured from previous point
    table = go.Table(
        domain=dict(x=[0, 1],
                    y=[0, 0.3]),
        columnwidth=[1, 2],
        # columnorder=[0, 1, 2,],
        header=dict(height=20,
                    values=[['<b>Date</b>'], ['<b>Actual Values </b>'], ['<b>% Change </b>'],
                            ],
                    font=dict(color=['rgb(45, 45, 45)'] * 5, size=14),
                    fill=dict(color='#d562be')),
        cells=dict(values=[df.round(3)[k].tolist() for k in ['Time', 'actuals', 'percentage_change']],
                   line=dict(color='#506784'),
                   align=['center'] * 5,
                   font=dict(color=['rgb(40, 40, 40)'] * 5, size=12),
                   # format = [None] + [",.4f"] + [',.4f'],
                   # suffix=[None] * 4,
                   suffix=[None] + [''] + [''] + ['%'] + [''],
                   height=27,
                   fill=dict(color=[test_df['anomaly_class'].map(color_map)],  # map based on anomaly level from dictionary
                             )
                   ))
    # Plot the actuals points
    Actuals = go.Scatter(name='Actuals',
                         x=dates,
                         y=df['actuals'],
                         xaxis='x1', yaxis='y1',
                         mode='line',
                         marker=dict(size=12,
                                     line=dict(width=1),
                                     color="blue"))
    # Highlight the anomaly points
    anomalies_map = go.Scatter(name="Anomaly",
                               showlegend=True,
                               x=dates,
                               y=anomaly_points,
                               mode='markers',
                               xaxis='x1',
                               yaxis='y1',
                               marker=dict(color="red",
                                           size=11,
                                           line=dict(
                                               color="red",
                                               width=2)))
    axis = dict(
        showline=True,
        zeroline=False,
        showgrid=True,
        mirror=True,
        ticklen=4,
        gridcolor='#ffffff',
        tickfont=dict(size=10))
    layout = dict(
        width=1000,
        height=865,
        autosize=False,
        title=metric_name,
        margin=dict(t=75),
        showlegend=True,
        xaxis1=dict(axis, **dict(domain=[0, 1],
                                 anchor='y1', showticklabels=True)),
        yaxis1=dict(axis, **dict(domain=[2 * 0.21 + 0.20, 1], anchor='x1', hoverformat='.2f')))
    fig = go.Figure(data=[table, anomalies_map, Actuals], layout=layout)
    plot(fig, filename='output/iqr/{0}.html'.format(metric_name), auto_open=False)


In [ ]:
def classify_iqr_anomalies(df, metric_name):
    df['metric_name'] = metric_name
    df = df.sort_values(by='Time', ascending=False)
    # Shift actuals by one Time to find the percentage chage between current and previous data point
    df['shift'] = df['actuals'].shift(-1)
    df['percentage_change'] = (
        (df['actuals'] - df['shift']) / df['actuals']) * 100
    
    # Categorise anomalies as 0-no anomaly, 1 - high anomaly
    df['anomaly_class'] = df['anomaly']
    max_anomaly_score = df['score'].loc[df['anomaly_class'] == 1].max()
    medium_percentile = df['score'].quantile(0.24)
    df['anomaly_class'].loc[(df['score'] > max_anomaly_score) & (
        df['score'] <= medium_percentile)] = 1
    return df

In [ ]:
# function to check if outlier lies between lower and upper bounds
def iqr_outlier_detection_col(x, colname):
    res = (x > iqr_df[iqr_df.column == colname]['upper_bound']) | (x < iqr_df[iqr_df.column == colname]['lower_bound'])
    return 1 if res.iloc[0] else 0

In [ ]:
# function to check if drift outside upper and lower bounds
def iqr_score(x, colname):
    row = iqr_df[iqr_df.column == colname]
    if(x > row['upper_bound'].iloc[0]):
        return (x - row['upper_bound']).iloc[0]
    elif(x < row['lower_bound'].iloc[0]):
        return (row['lower_bound'] - x).iloc[0]
    else:
        return 0

In [ ]:
for i in range(1, len(data_with_attacks.columns)-4):
    pred = data_with_attacks.iloc[:, i]\
    .apply(lambda x: iqr_outlier_detection_col(x, data_with_attacks.columns[i]))
    test_df = pd.DataFrame()
    test_df['Time'] = data_with_attacks['Time']
    
    # Find decision function to find the score and classify anomalies
    test_df['score'] = data_with_attacks.iloc[:, i]\
    .apply(lambda x: iqr_score(x, data_with_attacks.columns[i]))
    test_df['actuals'] = data_with_attacks.iloc[:, i]
    test_df['anomaly'] = pred
    
    test_df = classify_iqr_anomalies(test_df, data_with_attacks.columns[i])
    plot_iqr_anomaly(test_df, data_with_attacks.columns[i])

## References

1. https://towardsdatascience.com/anomaly-detection-with-isolation-forest-visualization-23cd75c281e2
2. https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html